In [ ]:
from Network import ensembleModel
from DB_Manage import DB_Bot
from DataLabeling import DataLabeling
from DataScaler import Data_StandardScaler
from Indicator import DataManage
import keras
import tensorflow as tf

In [ ]:
data = DB_Bot("BTC_USDT_30m").GetData()
parameter = [
        {"rsi" : {"period" : 14}},
        {"ma" : {"period" : 7}},
        {"ma" : {"period" : 25}},
        {"ema" :{"period" : 7}},
        {"ema" :{"period" : 25}},
        {"stochastic" : {"n" : 14,"m" : 5,"t" : 5}},
        {"bb" : {"length" : 21,"std" : 2}},
        {"kdj" : {}},
        {"macd" : {"fast_period": 12, "slow_period" : 26}}
]
DataManageBot = DataManage(data, parameter = parameter)
data = DataManageBot.get_data()
data = Data_StandardScaler(data.drop(["datetime"],axis=1))

In [ ]:
data.head()

In [ ]:
Labeler = DataLabeling(data, 20, "close")
Labeler.run()
Labeled_data = Labeler.data
Labeled_data.head()

In [ ]:
Labeled_data.label.value_counts()

In [ ]:
test_size = 10000
X,Y = Labeled_data.drop(["label"],axis=1), Labeled_data.label
x_train = X[:-test_size]
y_train = Y[:-test_size]
x_test = X[-test_size:]
y_test = Y[-test_size:]
print(x_train.shape, y_train.shape)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(256,input_shape = (x_train.shape[1],)))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(64,activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(32,activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(1,activation="sigmoid"))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                    metrics=["accuracy"],
                    loss = tf.keras.losses.BinaryCrossentropy())

In [ ]:
model.fit(x_train,y_train,epochs=10, batch_size=16)

In [16]:
pred = model.predict(x_test)
pred

array([[0.65271056],
       [0.5910429 ],
       [0.51872927],
       ...,
       [0.5603519 ],
       [0.51529545],
       [0.5689669 ]], dtype=float32)

In [17]:
result_label = []
cnt = 0

for i in range(len(pred)):
    if pred[i][0] < 0.2:
        result_label.append(0)
    elif pred[i][0] > 0.8:
        result_label.append(1)
    else:
        result_label.append(-1)
        
ck = 0 # 정답 카운트
cnt = 0 # 총 카운트
report_list = []
report_pred = []
for i in range(len(result_label)):
    if result_label[i] != -1:
        if result_label[i] == y_test.iloc[i]:
            ck += 1
        cnt += 1
print("accuracy :",round(ck/cnt * 100,2),"%")

accuracy : 79.67 %


In [20]:
from backtest import backtest
backtestBot = backtest(data, result_label, test_size, 0.002, 0.0008, 100000)
result = backtestBot.basicStrategy()

100%|██████████| 10000/10000 [00:07<00:00, 1275.70it/s]


In [19]:
for i in result:
    print(i,":",result[i])

averageNumberSales : 2476.244785276074
totalYield : -0.24557237069218374
win_rate : 0.0
MDD : -1.0
max_buying : 5074
NumberTrading : 4890


In [21]:
data.to_csv("debug.csv")

In [24]:
import pandas as pd
pd.DataFrame(result_label).to_csv("debugresultlabel.csv")

In [2]:
result_label = []
cnt = 0

for i in range(len(pred)):
    if pred[i][0] < 0.2:
        result_label.append(0)
    elif pred[i][0] > 0.8:
        result_label.append(1)
    else:
        result_label.append(-1)
        
ck = 0 # 정답 카운트
cnt = 0 # 총 카운트
report_list = []
report_pred = []
for i in range(len(result_label)):
    if result_label[i] != -1:
        if result_label[i] == y_test.iloc[i]:
            ck += 1
        cnt += 1
print("accuracy :",round(ck/cnt * 100,2),"%")

NameError: name 'pred' is not defined

In [5]:
from tqdm import tqdm
class backtest:
    def __init__(self,data, result_label, test_size, set_amount, fee, max_buy):
        self.data = data # 주가 데이터
        self.result_label = result_label # 머신러닝의 예측값
        self.test_size = test_size # 테스트 데이터의 길이
        self.set_amount = set_amount # 한번 매수할 떄 수량
        self.fee = fee # 한번 거래 할 때 수수료
        self.max_buy = max_buy # 최대 누적 매수 횟수

        # 손절형 순환매매에서 사용하는 변수
        self.buy_flag = 0
        self.sell_flag = 0

        # 공통 사용 변수
        self.quantityBuying = 0 # 현재 매수 수량
        self.BuyingList = [] # 현재 매수 했던 가격들의 리스트
        self.totalYield = 0 # 최종 수익률
        self.totalNumberSales = 0 # 총 매도 횟수
        self.win = 0
        self.quantityBuyingList = [] # 매수 수량을 저장하는 리스트
        self.amount = 0 # 현재 가지고 있는 주식 수량
        self.MDDList = [] # MDD를 계산하기 위한 리스트
        self.average_price = 0 # 평단가

        self.trading_history = []
    def initializes(self):
        self.buy_flag = 0
        self.sell_flag = 0

        # 공통 사용 변수
        self.quantityBuying = 0
        self.BuyingList = []
        self.totalYield = 0
        self.totalNumberSales = 0
        self.win = 0
        self.quantityBuyingList = []
        self.amount = 0
        self.MDDList = []
        self.average_price = 0

    def SellInitializes(self,currYield):
        if currYield > 0:
            self.win += 1
        self.average_price = 0
        self.totalNumberSales += 1
        self.totalYield += currYield
        self.amount = 0
        self.BuyingList.clear()
        self.quantityBuying = 0
        self.buy_flag = 0
        self.sell_flag = 0
        

    def basicStrategy(self):
        x_test = self.data.iloc[-self.test_size:]
        x_test['label'] = self.result_label
        self.initializes()
        for i in tqdm(range(len(x_test))):
            
            if x_test.iloc[i]['label'] == 1 and self.quantityBuying < self.max_buy:
                self.quantityBuying += 1
                self.average_price = ( self.average_price * self.amount + x_test.iloc[i]['close'] * self.set_amount) / (self.amount + self.set_amount)
                self.amount += self.set_amount
                self.BuyingList.append(x_test.iloc[i]['close'])

            if x_test.iloc[i]['label'] == 0 and self.quantityBuying != 0:
                currYield = (x_test.iloc[i]['close']-self.average_price) * self.amount - (self.average_price * self.fee * self.amount)
                self.quantityBuyingList.append(self.quantityBuying)

                self.SellInitializes(currYield)

            if self.quantityBuying > 0:
                self.MDDList.append(round((sum(self.BuyingList)/self.quantityBuying-x_test.iloc[i]['close'])/x_test.iloc[i]['close'],2))
        self.quantityBuyingList.sort()
        self.MDDList.sort()
        
        if len(self.quantityBuyingList) == 0 or self.totalNumberSales == 0:
            print("Zero DivisionError")
            raise ZeroDivisionError
        return {
            "averageNumberSales" : sum(self.quantityBuyingList)/len(self.quantityBuyingList),
            "totalYield" : self.totalYield,
            "win_rate" : self.win/self.totalNumberSales,
            "MDD" : self.MDDList[-1],
            "max_buying" : self.quantityBuyingList[-1],
            "NumberTrading" : self.totalNumberSales
        }
    
    def WaitingStrategy(self, term = 5):
        x_test = self.data.iloc[-self.test_size:]
        x_test['label'] = self.result_label
        self.initializes()

        waiting = 0
        for i in tqdm(range(len(x_test))):
            
            if x_test.iloc[i]['label'] == 1 and self.quantityBuying < self.max_buy and waiting > term:
                self.quantityBuying += 1
                self.average_price = ( self.average_price * self.amount + x_test.iloc[i]['close'] * self.set_amount) / (self.amount + self.set_amount)
                self.amount += self.set_amount
                self.BuyingList.append(x_test.iloc[i]['close'])
                self.trading_history.append(["buy", i, None])

            elif x_test.iloc[i]['label'] == 0 and self.quantityBuying != 0:
                currYield = (x_test.iloc[i]['close']-self.average_price) * self.amount - (self.average_price * self.fee * self.amount)
                self.quantityBuyingList.append(self.quantityBuying)

                self.SellInitializes(currYield)
                self.trading_history.append(["sell", i, currYield])
                waiting = 0

            if x_test.iloc[i]["label"] == 1:
                waiting += 1

            if self.quantityBuying > 0:
                self.MDDList.append(round((sum(self.BuyingList)/self.quantityBuying-x_test.iloc[i]['close'])/x_test.iloc[i]['close'],2))
        self.quantityBuyingList.sort()
        self.MDDList.sort()
        
        if len(self.quantityBuyingList) == 0 or self.totalNumberSales == 0:
            print("Zero DivisionError")
            raise ZeroDivisionError
        return {
            "averageNumberSales" : sum(self.quantityBuyingList)/len(self.quantityBuyingList),
            "totalYield" : self.totalYield,
            "win_rate" : self.win/self.totalNumberSales,
            "MDD" : self.MDDList[-1],
            "max_buying" : self.quantityBuyingList[-1],
            "NumberTrading" : self.totalNumberSales
        }
import pandas as pd
from DB_Manage import DB_Bot
data = DB_Bot("BTC_USDT_30m").GetData()
result_label = pd.read_csv("debugresultlabel.csv").to_numpy()[:,1].tolist()
object = backtest(data, result_label, 10000, 0.02, 0.0008, 10000)
print(object.WaitingStrategy(term=10))

/var/folders/5v/ywkv8vl52zs0xz8k0x15y8fm0000gn/T/ipykernel_58663/2578130533.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test['label'] = self.result_label
100%|██████████| 10000/10000 [00:05<00:00, 1902.24it/s]

{'averageNumberSales': 9.285714285714286, 'totalYield': 449.87576815999904, 'win_rate': 0.2857142857142857, 'MDD': 0.16, 'max_buying': 21, 'NumberTrading': 7}


In [9]:
trading_history = pd.DataFrame(object.trading_history,columns=["type", "index", "sell_price"])
trading_history

,type,index,sell_price
0,buy,1165,NaN
1,sell,1351,-0.373190
2,buy,2074,NaN
3,buy,2076,NaN
4,buy,2170,NaN
...,...,...,...
67,buy,8813,NaN
68,buy,8844,NaN
69,buy,8895,NaN
70,buy,8901,NaN


In [4]:
import sys, os,pandas as pd
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname("backend"))))))
data = pd.read_csv("/ML/BTC_USDT_1m.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/ML/BTC_USDT_1m.csv'